In [1]:
import dspy
from dspy.predict import Retry
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
import importlib
import json

importlib.reload(dspy)

# Configure the AzureOpenAI language model
azure_turbo = dspy.AzureOpenAI(api_base='https://bionlp-gpt4-wang.openai.azure.com/', api_key="a494edc84d714b6c8a12e7212974b793", api_version='2024-03-01-preview', model='gpt-4')
dspy.settings.configure(lm=azure_turbo)

/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LINE 127
System Prompt: None


In [2]:
class PubMedQA(dspy.Signature):
    question = dspy.InputField(desc="Question")
    context = dspy.InputField(desc="A list of facts which provide the necessary context to answer the question. You should directly mention the portions of the facts that you used to help answer the question.")
    options = dspy.InputField(desc="Options, you should select one of them based on whichever is correct.")
    answer = dspy.OutputField(desc="Your final answer should contain only one of YES, MAYBE, or NO (all in upper case) based on whichever option is correct")


class PubMedQA_Response(dspy.Module):
    """Use the provided context to answer a question by providing a step-by-step explanation and also respond with one of YES, MAYBE, or NO (all in upper case) based on whichever is correct"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(PubMedQA)

    def forward(self, question, context, options):
        response = self.generate_answer(context=context, question=question, options=options)

        '''
        while len(response.completions.answer) < 5:

            curr_len = len(response.completions.answer)

            for i in range(curr_len):
                response.completions.answer.append(response.completions.answer[i])
                response.completions.rationale.append(response.completions.rationale[i])

                if len(response.completions.answer) == 5:
                    break
        '''
        #best_response = self.compare_answers(context=context, question=question, options=options, completions=response.completions)
        
        #valid_response = "YES" in response.answer or "MAYBE" in response.answer or "NO" in response.answer
        #dspy.Suggest(valid_response, "Your final answer must be one of YES, MAYBE, or NO (all upper case) as part of your final answer")
        
        return dspy.Prediction(answer=response.answer, rationale=response.rationale)

In [3]:
import re

def eval_metric_pubmedqa(true, prediction, trace=None):
    try:
        pred = prediction.answer
        matches = re.findall(r"\b(YES|NO|MAYBE)\b", pred)
        parsed_answer = matches[-1] if matches else ""
        return parsed_answer == true.answer
    except:
        return False

In [4]:
import json 

with open("/Users/khandekarns/Documents/pubmedqa/data/test_set.json") as file:
    test_set = json.load(file)


with open("/Users/khandekarns/Documents/pubmedqa/data/test_ground_truth.json") as file:
    gt = json.load(file)


dspy_test_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in test_set.items():

    combined_context = ""

    for i, context in enumerate(test_set[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": test_set[key]['QUESTION'], "options": options, "context": combined_context, "answer": gt[key].upper()}).with_inputs("question", "context", "options") 
    dspy_test_set_pubmedqa.append(example)


In [5]:
import random

random.seed(42)

count = 0

with open("/Users/khandekarns/Documents/pubmedqa/data/ori_pqal.json") as file:
    val_data = json.load(file)

print(len(val_data))


dspy_val_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in val_data.items():

    if key in test_set:
        count += 1
        continue

    combined_context = ""

    for i, context in enumerate(val_data[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": val_data[key]['QUESTION'], "options": options, "context": combined_context, "answer": val_data[key]['final_decision'].upper()}).with_inputs("question", "context", "options") 

    dspy_val_set_pubmedqa.append(example)


# Shuffle the examples
random.shuffle(dspy_val_set_pubmedqa)

# Split into training and validation sets
dspy_train_set_pubmedqa = dspy_val_set_pubmedqa[:450]
dspy_val_set_pubmedqa = dspy_val_set_pubmedqa[450:500]

1000


In [6]:
from dspy.evaluate import Evaluate

evaluate_test = Evaluate(devset=dspy_test_set_pubmedqa, metric=eval_metric_pubmedqa, num_threads=3, display_progress=True, display_table=True)

In [7]:
evaluate_test(PubMedQA_Response())

Average Metric: 360 / 500  (72.0): 100%|██████████| 500/500 [09:07<00:00,  1.10s/it]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Is anorectal endosonography valuable in dyschesia?,1. YES 2. MAYBE 3. NO,1. Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the...,YES,YES,produce the answer. We can see from the context that anorectal endosonography was used in a study to demonstrate dysfunction of the anal sphincter and/or...,✔️ [True]
1,Is there a connection between sublingual varices and hypertension?,1. YES 2. MAYBE 3. NO,"1. Sublingual varices have earlier been related to ageing, smoking and cardiovascular disease. The aim of this study was to investigate whether sublingual varices are...",YES,YES,produce the answer. We know from the context that a study was conducted to investigate the relationship between sublingual varices and hypertension. The results of...,✔️ [True]
2,Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?,1. YES 2. MAYBE 3. NO,"1. Tacrolimus is a potent immunosuppressive drug used in organ transplantation. Because of its substantial toxic effects, narrow therapeutic index, and interindividual pharmacokinetic variability, therapeutic...",YES,YES,produce the answer. We know from the context that tacrolimus concentrations measured by the ACMIA method closely correlated with those measured by the MEIA method....,✔️ [True]
3,Does a physician's specialty influence the recording of medication history in patients' case notes?,1. YES 2. MAYBE 3. NO,1. To determine the impact of a physician's specialty on the frequency and depth of medication history documented in patient medical records. 2. A cross-sectional...,YES,YES,produce the answer. We can see from the context that a study was carried out to determine the impact of a physician's specialty on the...,✔️ [True]
4,Locoregional opening of the rodent blood-brain barrier for paclitaxel using Nd:YAG laser-induced thermo therapy: a new concept of adjuvant glioma therapy?,1. YES 2. MAYBE 3. NO,1. Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes. We address the question of whether LITT-induced locoregional...,YES,YES,produce the answer. We know from the context that Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes....,✔️ [True]
5,Spinal subdural hematoma: a sequela of a ruptured intracranial aneurysm?,1. YES 2. MAYBE 3. NO,1. A case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm is described. Such a case has...,YES,YES,produce the answer. We know from the context that a case of spinal subdural hematoma (SSDH) occurred following a subarachnoid hemorrhage (SAH) due to a...,✔️ [True]
6,Is there a correlation between androgens and sexual desire in women?,1. YES 2. MAYBE 3. NO,"1. For women, the correlation between circulating androgens and sexual desire is inconclusive. Substitution with androgens at physiological levels improves sexual function in women who...",YES,YES,produce the answer. We can see from the context that there is indeed a correlation between androgens and sexual desire in women. The context mentions...,✔️ [True]
7,Is the zeolite hemostatic agent beneficial in reducing blood loss during arterial injury?,1. YES 2. MAYBE 3. NO,1. Uncontrolled hemorrhage is the leading cause of fatality. The aim of this study was to evaluate the effect of zeolite mineral (QuikClot - Advanced...,YES,YES,produce the answer. We can see from the context that a study was conducted to evaluate the effect of the zeolite mineral (QuikClot - Advanced...,✔️ [True]
8,Are endothelial cell patterns of astrocytomas indicative of grade?,1. YES 2. MAYBE 3. NO,1. The most common primary brain tumors in childre

72.0

In [8]:
from tqdm import tqdm
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

tqdm._instances.clear()

config = dict(max_bootstrapped_demos=5, max_labeled_demos=0, num_candidate_programs=20, num_threads=6)
teleprompter = BootstrapFewShotWithRandomSearch(metric = eval_metric_pubmedqa, **config)
answer_question_teacher = teleprompter.compile(PubMedQA_Response(), trainset=dspy_train_set_pubmedqa, valset=dspy_val_set_pubmedqa)

Going to sample between 1 and 5 traces per predictor.
Will attempt to bootstrap 20 candidate sets.


Average Metric: 36 / 50  (72.0): 100%|██████████| 50/50 [00:36<00:00,  1.38it/s]


Score: 72.0 for set: [0]
New best sscore: 72.0 for seed -3
Scores so far: [72.0]
Best score: 72.0


Average Metric: 36 / 50  (72.0): 100%|██████████| 50/50 [00:00<00:00, 431.38it/s]


Score: 72.0 for set: [0]
Scores so far: [72.0, 72.0]
Best score: 72.0


...

Score: 78.0 for set: [2]
Scores so far: [72.0, 72.0, 80.0, 80.0, 72.0, 72.0, 78.0, 68.0, 84.0, 80.0, 82.0, 74.0, 78.0, 80.0, 80.0, 74.0, 82.0, 84.0, 76.0, 76.0, 80.0, 78.0]
Best score: 84.0
Average of max per entry across top 1 scores: 0.84
Average of max per entry across top 2 scores: 0.88
Average of max per entry across top 3 scores: 0.88
Average of max per entry across top 5 scores: 0.9
Average of max per entry across top 8 scores: 0.9
Average of max per entry across top 9999 scores: 0.9


  0%|          | 1/450 [00:04<31:19,  4.19s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


Average Metric: 37 / 50  (74.0): 100%|██████████| 50/50 [00:42<00:00,  1.16it/s]

Score: 74.0 for set: [1]
Scores so far: [72.0, 72.0, 80.0, 80.0, 72.0, 72.0, 78.0, 68.0, 84.0, 80.0, 82.0, 74.0, 78.0, 80.0, 80.0, 74.0, 82.0, 84.0, 76.0, 76.0, 80.0, 78.0, 74.0]
Best score: 84.0
Average of max per entry across top 1 scores: 0.84
Average of max per entry across top 2 scores: 0.88
Average of max per entry across top 3 scores: 0.88
Average of max per entry across top 5 scores: 0.9
Average of max per entry across top 8 scores: 0.9
Average of max per entry across top 9999 scores: 0.9
23 candidate programs found.


In [9]:
evaluate_test(answer_question_teacher)

Average Metric: 367 / 500  (73.4): 100%|██████████| 500/500 [10:02<00:00,  1.20s/it]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Is anorectal endosonography valuable in dyschesia?,1. YES 2. MAYBE 3. NO,1. Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the...,YES,YES,produce the answer. We know from the context that the study was conducted to demonstrate dysfunction of the anal sphincter and/or the musculus (m.) puborectalis...,✔️ [True]
1,Is there a connection between sublingual varices and hypertension?,1. YES 2. MAYBE 3. NO,"1. Sublingual varices have earlier been related to ageing, smoking and cardiovascular disease. The aim of this study was to investigate whether sublingual varices are...",YES,YES,produce the answer. We know from the context that the study was conducted to investigate whether sublingual varices are related to the presence of hypertension....,✔️ [True]
2,Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?,1. YES 2. MAYBE 3. NO,"1. Tacrolimus is a potent immunosuppressive drug used in organ transplantation. Because of its substantial toxic effects, narrow therapeutic index, and interindividual pharmacokinetic variability, therapeutic...",YES,MAYBE,"produce the answer. We know from the context that the study was conducted to investigate the comparability of the results of 2 immunoassay systems, affinity...",False
3,Does a physician's specialty influence the recording of medication history in patients' case notes?,1. YES 2. MAYBE 3. NO,1. To determine the impact of a physician's specialty on the frequency and depth of medication history documented in patient medical records. 2. A cross-sectional...,YES,YES,produce the answer. We know from the context that a study was conducted to determine the impact of a physician's specialty on the frequency and...,✔️ [True]
4,Locoregional opening of the rodent blood-brain barrier for paclitaxel using Nd:YAG laser-induced thermo therapy: a new concept of adjuvant glioma therapy?,1. YES 2. MAYBE 3. NO,1. Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes. We address the question of whether LITT-induced locoregional...,YES,YES,produce the answer. We know from the context that Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes....,✔️ [True]
5,Spinal subdural hematoma: a sequela of a ruptured intracranial aneurysm?,1. YES 2. MAYBE 3. NO,1. A case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm is described. Such a case has...,YES,MAYBE,produce the answer. We know from the context that a case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) due to a ruptured internal...,False
6,Is there a correlation between androgens and sexual desire in women?,1. YES 2. MAYBE 3. NO,"1. For women, the correlation between circulating androgens and sexual desire is inconclusive. Substitution with androgens at physiological levels improves sexual function in women who...",YES,YES,produce the answer. We know from the context that the study was conducted to investigate a possible correlation between serum levels of androgens and sexual...,✔️ [True]
7,Is the zeolite hemostatic agent beneficial in reducing blood loss during arterial injury?,1. YES 2. MAYBE 3. NO,1. Uncontrolled hemorrhage is the leading cause of fatality. The aim of this study was to evaluate the effect of zeolite mineral (QuikClot - Advanced...,YES,YES,produce the answer. We know from the context that the study was conducted to evaluate the effect of zeolite mineral (QuikClot - Advanced Clotting Sponge...,✔️ [True]
8,Are endothelial cell patterns of astrocytomas indicative of grade?,1. YES 2. MAYBE 3. NO,1. The most common pr

73.4

In [11]:
from dspy.teleprompt import MIPROv2
import random


teleprompter = MIPROv2(prompt_model= azure_turbo, task_model=azure_turbo, metric=eval_metric_pubmedqa, num_candidates=20, metric_threshold=None)
kwargs = dict(num_threads=3, display_progress=True, display_table=True)
compiled_prompt_opt = teleprompter.compile(PubMedQA_Response(), trainset=dspy_train_set_pubmedqa, valset= dspy_val_set_pubmedqa, max_bootstrapped_demos=5, max_labeled_demos=0, eval_kwargs=kwargs, requires_permission_to_run=False)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 20 * 1 lm calls in program + (2) lm calls in program aware proposer = 32 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 450 examples in train set * 3 full evals = 2100 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may

  0%|          | 1/450 [00:01<14:34,  1.95s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


  2%|▏         | 8/450 [00:09<09:10,  1.25s/it]


Bootstrapped 5 full traces after 9 examples in round 0.


...

FULL TRACE


Average Metric: 1 / 1  (100.0): 100%|██████████| 1/1 [00:02<00:00,  2.66s/it]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Juvenile ankylosing spondylitis--is it the same disease as adult ankylosing spondylitis?,1. YES 2. MAYBE 3. NO,1. Juvenile and adult forms of ankylosing spondylitis (AS) have been shown to have different clinical presentation and outcome in Caucasians. We did this retrospective...,NO,NO,produce the answer. The context provided indicates that juvenile and adult forms of ankylosing spondylitis (AS) have different clinical presentations and outcomes. This is evident...,✔️ [True]


[I 2024-07-29 22:58:52,316] Trial 29 finished with value: 88.0 and parameters: {'0_predictor_instruction': 12, '0_predictor_demos': 11}. Best is trial 2 with value: 92.0.





Considering the 'question', assess the official documents/studies/lab data provided as 'context', and scrutinize how well each 'option' aligns with the interpretations made from the 'context', understanding and incorporating the specialized medical terminologies & complex biological logic relevant to the scenario described in 'context'. Make a selection among either 'YES', 'NO' or 'MAYBE' as your 'answer', by creating a well-substantiated sequence of findings and label it 'rationale' that strategically rationalizes merits/limitations scattered among options. When writing your answer and rationale, reflect your depth analysis by highlighting and weighing the core information, cropping methodically from 'context', that explicitly confers this choice. Your select 'answer' should precisely continent

---

Follow the following format.

Question: Question

Context: A list of facts which provide the necessary context to answer the question. You should directly mention the portions of the f

In [12]:
evaluate_test(compiled_prompt_opt)

Average Metric: 373 / 500  (74.6): 100%|██████████| 500/500 [09:57<00:00,  1.19s/it]


,question,options,context,example_answer,pred_answer,rationale,eval_metric_pubmedqa
0,Is anorectal endosonography valuable in dyschesia?,1. YES 2. MAYBE 3. NO,1. Dyschesia can be provoked by inappropriate defecation movements. The aim of this prospective study was to demonstrate dysfunction of the anal sphincter and/or the...,YES,YES,produce the answer. We can see from the context that a study was conducted to demonstrate dysfunction of the anal sphincter and/or the musculus (m.)...,✔️ [True]
1,Is there a connection between sublingual varices and hypertension?,1. YES 2. MAYBE 3. NO,"1. Sublingual varices have earlier been related to ageing, smoking and cardiovascular disease. The aim of this study was to investigate whether sublingual varices are...",YES,YES,produce the answer. We can see from the context that a study was conducted to investigate whether sublingual varices are related to the presence of...,✔️ [True]
2,Is the affinity column-mediated immunoassay method suitable as an alternative to the microparticle enzyme immunoassay method as a blood tacrolimus assay?,1. YES 2. MAYBE 3. NO,"1. Tacrolimus is a potent immunosuppressive drug used in organ transplantation. Because of its substantial toxic effects, narrow therapeutic index, and interindividual pharmacokinetic variability, therapeutic...",YES,MAYBE,"produce the answer. We can see from the context that a study was conducted to compare the results of two immunoassay systems, the affinity column-mediated...",False
3,Does a physician's specialty influence the recording of medication history in patients' case notes?,1. YES 2. MAYBE 3. NO,1. To determine the impact of a physician's specialty on the frequency and depth of medication history documented in patient medical records. 2. A cross-sectional...,YES,YES,produce the answer. We can see from the context that a study was conducted to determine the impact of a physician's specialty on the frequency...,✔️ [True]
4,Locoregional opening of the rodent blood-brain barrier for paclitaxel using Nd:YAG laser-induced thermo therapy: a new concept of adjuvant glioma therapy?,1. YES 2. MAYBE 3. NO,1. Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability changes. We address the question of whether LITT-induced locoregional...,YES,YES,produce the answer. We can see from the context that Nd:YAG laser-induced thermo therapy (LITT) of rat brains is associated with blood-brain barrier (BBB) permeability...,✔️ [True]
5,Spinal subdural hematoma: a sequela of a ruptured intracranial aneurysm?,1. YES 2. MAYBE 3. NO,1. A case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) because of a ruptured internal carotid aneurysm is described. Such a case has...,YES,MAYBE,produce the answer. We can see from the context that a case of spinal subdural hematoma (SSDH) following subarachnoid hemorrhage (SAH) due to a ruptured...,False
6,Is there a correlation between androgens and sexual desire in women?,1. YES 2. MAYBE 3. NO,"1. For women, the correlation between circulating androgens and sexual desire is inconclusive. Substitution with androgens at physiological levels improves sexual function in women who...",YES,YES,"produce the answer. We can see from the context that the study found correlations between sexual desire and various androgens in women. Specifically, sexual desire...",✔️ [True]
7,Is the zeolite hemostatic agent beneficial in reducing blood loss during arterial injury?,1. YES 2. MAYBE 3. NO,1. Uncontrolled hemorrhage is the leading cause of fatality. The aim of this study was to evaluate the effect of zeolite mineral (QuikClot - Advanced...,YES,YES,produce the answer. We can see from the context that the application of the zeolite mineral (QuikClot - Advanced Clotting Sponge [QC-ACS]) led to a...,✔️ [True]
8,Are endothelial cell patterns of astrocytomas indicative of grade?,1. YES 2. MAYBE 3. NO,1. The most common primary brain tumors in children and a

74.6